# (day2) Section 3: 過学習

本書は、「深層学習前編（day2）レポート」の、「Section 3: 過学習」についてのものです。


## 1. 要点まとめ


### 過学習

学習の結果としての訓練誤差は、学習が進むにつれて小さくなる。<br>
対して、予測の結果であるテスト誤差が、学習が進んでも小さくならない、または、大きくなってしまう現象。

学習の対象である訓練データに過度に適合し、未知のデータの予測に役立たない。

原因と、本章での対応方法は以下の通り。

| 原因 | 対応方法 |
| -- | -- |
| パラメータの値が適切ではない<br>入力値に対して NN が大き過ぎる | L1、L2正則化でパラメータの値を制約 |
| ノードが多い | ドロップアウトでノードの数を制約 |

- NOTE: ドロップアウトは、一時的に削除するだけなのでは?
- NOTE: パラメータの数が多いことと、層数が原因の場合の対応方法は?<br>
  または、対応はそれぞれの数を減らすこととして、原因であることを検知する方法は?


### 正則化


#### Weight decay ( 荷重減衰 )

重みのばらつき ( 学習の成果 ) を維持しつつ、過学習原因となる大き過ぎる重みを抑制すること。

講義「機械学習」/「2. 非線形回帰モデル」にも登場した、 L1正則化、L2正則化を使用する。


#### L1 、 L2 正則化


##### 数式

誤差関数に、正則化項(罰則化項)として、 p ノルムを加える。

$$
E_n(\textbf{w}) + \frac{1}{p} \lambda \Vert x \Vert_p
$$

p ノルムの計算は、以下の通り。

$$
\Vert x \Vert_p = (
  \vert x_1 \vert^p + \ldots + \vert x_n \vert^p
)^{\frac{1}{p}}
$$

- p = 1 の場合、 L1 ノルムを使った L1 正則化 ( Lasso 回帰、マンハッタン距離を使用 )
- p = 2 の場合、 L2 ノルムを使った L2 正則化 ( Ridge 回帰、ユークリッド距離を使用 )


##### NN の中での適用箇所

p ノルムは誤差関数に適用する。(再掲)

$$
E_n(\textbf{w}) + \frac{1}{p} \lambda \Vert x \Vert_p
$$

p ノルムは、各層の重みから計算する。 ( 例として、2層 )

$$
\Vert x \Vert_p = \Vert \textbf{W}^{(1)} \Vert_p
+
\Vert \textbf{W}^{(2)} \Vert_p
$$

$$
\Vert \textbf{W}^{(1)} \Vert_p = (
  \vert \textbf{W}^{(1)}_1 \vert^p + \ldots + \vert \textbf{W}^{(1)}_n \vert^p
)^{\frac{1}{p}}
$$

$$
\Vert \textbf{W}^{(2)} \Vert_p = (
  \vert \textbf{W}^{(2)}_1 \vert^p + \ldots + \vert \textbf{W}^{(2)}_n \vert^p
)^{\frac{1}{p}}
$$


##### 対応するコード

`2_5_overfiting.ipynb` の、 `## weight decay` の下のセル。

`### L2` の場合、

ノルムの計算は、

```python
    for idx in range(1, hidden_layer_num+1):
        ...
        weight_decay += 0.5 * weight_decay_lambda * np.sqrt(np.sum(network.params['W' + str(idx)] ** 2))
```

誤差関数への適用は、

```python
    loss = network.loss(x_batch, d_batch) + weight_decay
```

`### L1` の場合、ノルムの計算が以下の様に変わる。重みを2乗しない。

```python
    for idx in range(1, hidden_layer_num+1):
        ...
        weight_decay += weight_decay_lambda * np.sum(np.abs(network.params['W' + str(idx)]))
```


##### L1 、 L2 正則化の特徴

以下の通り。

| 正則化の種類 | 推定量 | 正則化(罰則)項 | 誤差の等高線グラフとの接点 | 重みの推定 |
| -- | -- | -- | -- | -- |
| L1 正則化 | Lasso | L1 ノルム | $w_0 + w_1 \leq 1$: 四角の頂点と接しやすい | スハース推定 ( 0 の要素の割合 ) |
| L2 正則化 | Ridge | L2 ノルム | $w_0^2 + w_1^2 \leq 1$: 円の接線と接する | 縮小推定 ( 要素を 0 に近づける ) |


#### ドロップアウト

ランダムにノードを削除し、データを変化させずに異なるモデルを学習していると NN に解釈させる手法。

- NOTE: page. 69 のデータ拡張は、このドロップアウトとはまた別の方法。


##### コード

`lesson_2/multi_layer_net.py` の、 `class MultiLayerNet` の `__init__` で、
ドロップアウトする割合を設定して、ドロップアウトの層を追加している。

```python
            if self.use_dropout:
                self.layers['Dropout' + str(idx)] = layers.Dropout(dropout_ratio)
```

ドロップアウトな設定は、
`common/optimizer.py` の、 `class Dropout` に設定される。

```python
    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask
```

学習の、前伝播にドロップアウトするノードをマスクで決定して、使用するノードを抑制する。
学習の逆伝播 ( 誤差関数計算時 ) と、予測時にも前伝播の時に決めたマスクでノードを抑制する。

マスクは、イテレーションを試行する度に変わるので、それに合わせてドロップアウトされるノードが変わる。よって、同じデータで異なるモデルを学習させる効果がある。

イテレーションの試行をしているのは、 `lesson_2/2_5_overfiting.ipynb` の、
`## Dropout` から 2つ下のあるセル。

```python
for i in range(iters_num):
...
    loss = network.loss(x_batch, d_batch)
...
    
    if (i+1) % plot_interval == 0:
        accr_train = network.accuracy(x_train, d_train)
        accr_test = network.accuracy(x_test, d_test)
...
```


## 2. 確認テスト

以降の "page. " は、講義資料のページの番号です。


### Page. 59

以下を最小化する回帰をリッジ回帰。

$$
E(\textbf{w}) = \Vert \textbf{y} - \hat{\textbf{X}}\textbf{w} \Vert^2
+
\lambda \Vert \textbf{w} \Vert^2
$$

$ \lambda = 0 $ とすると、

$$
E(\textbf{w}) = \Vert \textbf{y} - \hat{\textbf{X}}\textbf{w} \Vert^2
$$

として、二乗和誤差を最小化する線形回帰となる。
しかし、解答の "(b)" は、「非線形回帰」。選択できなかった。

正則化項について、

$$
\lambda \Vert \textbf{w} \Vert^2
$$

$ \lambda $ を大きな値にすると、全体を最小化しなくてはならないから、
$ \textbf{w} $ は小さくならなくてはならない。<br>
また、 "限りなく 0 に近づく" であって、 0 にはならない。
page. 64 のグラフの、左側 ( Ridge 推定量 ) の通り、誤差の等高線と制約の領域(円)との接線が、縦軸、横軸の位置にならない、つまり、縦方向、横方向のパラメータが 0 にならないため。
( 右側の Lasso では制約の四角形の頂点で接するので、グラフの例では、縦軸に接して、横方向のパラメータが 0 になりえる。 )<br>
よって、 "(a)" を選択。

- NOTE: [超入門！リッジ回帰・Lasso回帰・Elastic Netの基本と特徴をサクッと理解！ | AIZINE（エーアイジン）](https://aizine.ai/ridge-lasso-elasticnet/)


### Page. 64

Lasso 推定量で、スパース推定である、右側のグラフである。<br>
誤差の等高線が、制約を表す四角の頂点と接しやすい

- NOTE: [PRMLの線形回帰モデル（線形基底関数モデル）](https://www.slideshare.net/yasunoriozaki12/prml-29439402)
